In [12]:
import pandas as pd
import numpy as np
import json
from datetime import date, timedelta
#from sodapy import Socrata
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import urllib.request
from selenium.webdriver.common.keys import Keys
import os

In [2]:
date_2year_ago = (date.today() - timedelta(days=2*365))
date_today = date.today()
print('Las fechas a utilizar son: fecha dos anios atras: ' + str(date_2year_ago) + ' y fecha actual: ' + str(date_today))
# Cambiar esta variable si se necesita una fecha especifica
date_2year_ago = 20190000#int(date_2year_ago.strftime('%Y%m%d'))
date_today = int(date_today.strftime('%Y%m%d'))

Las fechas a utilizar son: fecha dos anios atras: 2019-04-27 y fecha actual: 2021-04-26


In [3]:
def first_df():
    # leemos desde la URL el json que contiene la tabla de informacion acerca de las tasas
    df = pd.read_json('https://www.datos.gov.co/resource/mcec-87by.json')
    # hacemos transformaciones a las fechas para tenerlas YYYYMMDD numericas
    df['vigenciadesde'] = pd.to_datetime(df['vigenciadesde']).dt.strftime('%Y%m%d').astype('int32')
    df['vigenciahasta'] = pd.to_datetime(df['vigenciahasta']).dt.strftime('%Y%m%d').astype('int32')
    # retornamos el data frame desde la fecha 2019
    return df[df['vigenciadesde'] >= date_2year_ago]

In [4]:
df  = first_df()
df

,valor,unidad,vigenciadesde,vigenciahasta
0,3659.62,COP,20210427,20210427
1,3640.07,COP,20210424,20210426
2,3630.81,COP,20210423,20210423
3,3639.12,COP,20210422,20210422
4,3636.26,COP,20210421,20210421
...,...,...,...,...
547,3128.07,COP,20190110,20190110
548,3164.75,COP,20190109,20190109
549,3208.56,COP,20190105,20190108
550,3241.20,COP,20190104,20190104


In [5]:
# Se agrupa por anio y mes. Se cuenta para saber cuantos registros hay por cada mes lo que equivale
# a la cantidad de dias por mes
tasa_spot = first_df()
periodo = (tasa_spot['vigenciadesde']/100).astype('int32')
data = periodo.to_list()
names = ['periodo']
new_df = pd.DataFrame(data, columns=names)
new_df.value_counts().sort_index()

periodo
201901     19
201902     19
201903     21
201904     19
201905     21
201906     19
201907     20
201908     21
201909     19
201910     22
201911     19
201912     20
202001     19
202002     20
202003     20
202004     20
202005     20
202006     18
202007     22
202008     19
202009     21
202010     22
202011     16
202012     21
202101     18
202102     19
202103     21
202104     17
dtype: int64

In [6]:
# Enter the information from those sections here
#socrata_domain = 'datos.gov.co'
#socrata_dataset_identifier = 'mcec-87by'

# App Tokens can be generated by creating an account at https://opendata.socrata.com/signup
# Tokens are optional (`None` can be used instead), though requests will be rate limited.
#
# If you choose to use a token, run the following command on the terminal (or add it to your .bashrc)
# $ export SODAPY_APPTOKEN=<token>
#credentials = json.load(open('credentials.json'))
#socrata_token = credentials['TOKEN']


In [7]:
#client = Socrata(socrata_domain, socrata_token)
#print("Domain: {domain:}\nSession: {session:}\nURI Prefix: {uri_prefix:}".format(**client.__dict__))

In [8]:
#results = client.get(socrata_dataset_identifier)
#df = pd.DataFrame.from_dict(results)
#df_splt = df['vigenciadesde'].str.split('-')
#names = ['anio', 'mes', 'dia']
#data2 = df_splt.to_list()
#new_df2 = pd.DataFrame(data2, columns=names)
#new_df2.groupby(['anio', 'mes']).count()

In [9]:
# Segundo dataframe
# Se configuran las opciones del navegador para que empiece maximixado y sin extensiones habilitadas
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

In [10]:
# Funcion para eliminar duplicados de una lista
def remove_duplicate_list(lst):
    ''' Funcion para eliminar duplicados de una lista'''
    return list(dict.fromkeys(lst))

# Se crea instancia del driver que abrira el navegador web con las opciones definidas anteriormente
# Cambiar direccion a donde se encuentra el driver
driver_path = r'C:\Users\Efra\Downloads\chromedriver_win32\chromedriver.exe'
driver = webdriver.Chrome(driver_path, options = options)
# Se le pasa el link a abrir en el navegador
driver.get('https://totoro.banrep.gov.co/analytics/saw.dll?Portal&PortalPath=%2Fshared%2FDashboards_T%2FD_Estad%C3%ADsticas%2FEstad%C3%ADsticas&NQUser=publico&NQPassword=publico123&lang=es&page=Tasas%20de%20inter%C3%A9s%20y%20sector%20financiero&pagina=Otras%20tasas%20de%20inter%C3%A9s')

WebDriverWait(driver, 15)
driver.switch_to.frame(driver.find_element_by_id('frame_dashboard'))

# Se da click en el boton otras tasas de interes
WebDriverWait(driver, 15)\
.until(EC.element_to_be_clickable((By.XPATH,
                                      '/html/body/div[7]/div/div/table/tbody/tr/td[1]/div/div/table/tbody/tr/td[2]/div/table/tbody/tr/td[2]/table/tbody/tr/td/div')))\
    .click()

# Se da click en el menu desplegable Indicador bancario de referencia IBR
WebDriverWait(driver, 15)\
.until(EC.element_to_be_clickable((By.CLASS_NAME,
                                      'sui-treeview-item-toggle-collapsed')))
driver.find_elements_by_class_name('sui-treeview-item-toggle-collapsed')[1].click()


#print(len(driver.find_elements_by_xpath("//a[@href]")))
# Se guardan todos los links asocaidos al atributo o etiqueta href
href = driver.find_elements_by_xpath("//a[@href]")
href_list = []
# Se recorre la lista href en busca de los links que pertenezcan a los insumos de IBR
for i in href:
    try:
        contains = str(i.get_attribute('href')).replace('%20', '').upper()
        #print(contains)
        # Si el href contiene alguno de los strings, es guardado en una lista que contendra los enlaces a descargar
        if('IBR_PLAZOOVERNIGHTNOMINALPARAUNRANGODEFECHASDADOIQY' in contains 
           or 'IBR_PLAZOUNMESNOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           or 'IBR_PLAZOTRESMESESNOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           or 'IBR_PLAZOMESVENCIDONOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           or 'IBR_PLAZOSEISMESESNOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           and 'DOWNLOAD' in contains):
            src = i.get_attribute('href')
            href_list.append(src)
            #print('saca algo ' + src)
    except:
        #print(error)
        print('error href')

# Como la lista posiblemente guarde enlaces duplicados, se aplica la funcion para remover duplicados y asi descargar una sola vez
# cada archivo
href_list = remove_duplicate_list(href_list)
print('Cantidad de archivos a descargar: ' + str(len(href_list)))
for j in href_list:
    try:
        driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + 't') 
        driver.get(j)
        print('Archivo descargado asociado a: ' + j)
    except AssertionError as error:
        print(error)
        print('error link')
        
# Cambiar a 60 segundos para que descargue antes de cerrar
time.sleep(30)
driver.close()

error href
error href
Cantidad de archivos a descargar: 5
Archivo descargado asociado a: https://totoro.banrep.gov.co/analytics/saw.dll?Download&Format=excel2007&Extension=.xls&BypassCache=true&lang=es&path=%2Fshared%2FSeries%20Estad%C3%ADsticas_T%2F1.%20IBR%2F%201.1.IBR_Plazo%20overnight%20nominal%20para%20un%20rango%20de%20fechas%20dado%20IQY
Archivo descargado asociado a: https://totoro.banrep.gov.co/analytics/saw.dll?Download&Format=excel2007&Extension=.xls&BypassCache=true&lang=es&path=%2Fshared%2FSeries%20Estad%C3%ADsticas_T%2F1.%20IBR%2F%201.2.IBR_Plazo%20un%20mes%20nominal%20para%20un%20rango%20de%20fechas%20dado%20IQY
Archivo descargado asociado a: https://totoro.banrep.gov.co/analytics/saw.dll?Download&Format=excel2007&Extension=.xls&BypassCache=true&lang=es&path=%2Fshared%2FSeries%20Estad%C3%ADsticas_T%2F1.%20IBR%2F%201.3.IBR_Plazo%20tres%20meses%20nominal%20para%20un%20rango%20de%20fechas%20dado%20IQY
Archivo descargado asociado a: https://totoro.banrep.gov.co/analytics/saw

# Tratamiento archivos IBR

In [3]:
def clean_ibr_df(ruta):
    '''Funcion que limpia y adecua los archivos IBR, como parametro recibe la ruta del archivo excel a tratar'''
    # Limpieza y adecuacion primer data frame IBR plazo overnight
    # Se lee la ruta donde se encuentra el archivo excel IBR y se le indica que los encabezados estaran en la posicion o celda 8
    # y que omita los pie de pagina
    df = pd.read_excel(ruta, header = 8, skipfooter = 7)
    # se renombran las columnas para mas claridad en el df
    df.rename(columns = {'Fecha (dd/mm/aaaa)' : 'Fecha', 'IBR' : 'IBR_efectivo', 'IBR.1' : 'IBR_nominal'}, inplace = True)
    # Se valida si la columna plazo existe, si no existe automaticamente le inserta una nueva columna al df y la rellena con el valor 0
    if 'Plazo real (días calendario)' not in df.columns:
        df['Plazo real (días calendario)'] = 0
    # Se toman solo esas 4 columnas del df
    df = df[['Fecha', 'Plazo real (días calendario)', 'IBR_efectivo', 'IBR_nominal']]
    # Se le hacen adecuaciones a la fecha para dejarla en formato YYYYMMDD numerico
    df['Fecha'] = pd.to_datetime(df['Fecha']).dt.strftime('%Y%m%d').astype('int32')
    # Se convierten las tasas de tipo texto a decimal
    df.IBR_efectivo.replace({',':'.'}, regex = True, inplace = True)
    df['IBR_efectivo'] = df['IBR_efectivo'].astype(float)
    df.IBR_nominal.replace({',':'.'}, regex = True, inplace = True)
    df['IBR_nominal'] = df['IBR_nominal'].astype(float)
    # Se rellena con 0 las tasas que esten en blanco o nulas
    df['IBR_efectivo'].fillna(0, inplace = True)
    df['IBR_nominal'].fillna(0, inplace = True)

    return df

In [8]:
def nominal_to_efectivo(tasa):
    '''Funcion para convertir tasa nominal a efectiva'''
    tasa = tasa / 100
    tasa_efectiva = (((1 + (tasa / 12))**12) - 1) * 100
    return tasa_efectiva

In [15]:
ibr_overnight = clean_ibr_df(r'C:\Users\Efra\Downloads\1.1.IBR_Plazo overnight nominal para un rango de fechas dado IQY.xlsx')
ibr_overnight

,Fecha,Plazo real (días calendario),IBR_efectivo,IBR_nominal
0,20210426,1.0,1.749,1.710
1,20210423,3.0,1.748,1.709
2,20210422,1.0,1.747,1.708
3,20210421,1.0,1.742,1.703
4,20210420,1.0,1.739,1.700
...,...,...,...,...
3241,20080109,NaN,9.520,8.970
3242,20080108,NaN,9.509,8.960
3243,20080104,NaN,9.495,8.951
3244,20080103,NaN,9.520,8.970


In [4]:
ibr_mes = clean_ibr_df(r'C:\Users\Efra\Downloads\1.2.IBR_Plazo un mes nominal para un rango de fechas dado IQY.xlsx')
ibr_mes

,Fecha,Plazo real (días calendario),IBR_efectivo,IBR_nominal
0,20210426,30,1.749,1.711
1,20210423,31,1.748,1.710
2,20210422,32,1.748,1.710
3,20210421,30,1.746,1.708
4,20210420,30,1.737,1.700
...,...,...,...,...
2122,20120808,33,5.098,4.915
2123,20120806,31,5.104,4.920
2124,20120803,31,5.104,4.920
2125,20120802,32,5.093,4.910


In [5]:
ibr_trimestral = clean_ibr_df(r'C:\Users\Efra\Downloads\1.3.IBR_Plazo tres meses nominal para un rango de fechas dado IQY.xlsx')
ibr_trimestral

,Fecha,Plazo real (días calendario),IBR_efectivo,IBR_nominal
0,20210426,91,1.749,1.714
1,20210423,91,1.746,1.711
2,20210422,91,1.743,1.708
3,20210421,91,1.743,1.708
4,20210420,92,1.740,1.705
...,...,...,...,...
2122,20120808,92,4.898,4.745
2123,20120806,92,4.903,4.750
2124,20120803,95,4.913,4.760
2125,20120802,92,4.914,4.760


In [6]:
ibr_mes_nominal = clean_ibr_df(r'C:\Users\Efra\Downloads\1.4.IBR_Plazo mes vencido nominal para un rango de fechas dado IQY.xlsx')
ibr_mes_nominal

,Fecha,Plazo real (días calendario),IBR_efectivo,IBR_nominal
0,20120731,0,0.0,5.015
1,20120724,0,0.0,5.161
2,20120717,0,0.0,5.160
3,20120710,0,0.0,5.145
4,20120703,0,0.0,5.098
...,...,...,...,...
234,20080205,0,0.0,9.073
235,20080129,0,0.0,9.055
236,20080122,0,0.0,9.070
237,20080115,0,0.0,9.078


In [9]:
ibr_mes_nominal['IBR_efectivo'] = ibr_mes_nominal['IBR_nominal'].apply(nominal_to_efectivo)
ibr_mes_nominal

,Fecha,Plazo real (días calendario),IBR_efectivo,IBR_nominal
0,20120731,0,5.131893,5.015
1,20120724,0,5.284849,5.161
2,20120717,0,5.283800,5.160
3,20120710,0,5.268076,5.145
4,20120703,0,5.218822,5.098
...,...,...,...,...
234,20080205,0,9.459970,9.073
235,20080129,0,9.440416,9.055
236,20080122,0,9.456710,9.070
237,20080115,0,9.465402,9.078


In [11]:
ibr_semestral = clean_ibr_df(r'C:\Users\Efra\Downloads\1.5.IBR_Plazo seis meses nominal para un rango de fechas dado IQY.xlsx')
ibr_semestral

,Fecha,Plazo real (días calendario),IBR_efectivo,IBR_nominal
0,20210426,183,1.847,1.813
1,20210423,185,1.849,1.815
2,20210422,183,1.849,1.815
3,20210421,183,1.841,1.808
4,20210420,183,1.830,1.797
...,...,...,...,...
1195,20160527,185,7.411,7.181
1196,20160526,186,7.412,7.182
1197,20160525,184,7.401,7.170
1198,20160524,184,7.406,7.175


In [ ]:
#r.describe()
#pd.set_option('display.min_rows', None)

# Tratamiento archivos Ice Libor

In [38]:
# Leemos la ruta donde se encuentra el proyecto
path = os.getcwd()
# Le anadimos la parte de los insumos
dir_insumos = path + '\Insumos\libor\\'
dir_insumos

'D:\\Proyectos\\proyecto_hector\\Insumos\\libor\\'

In [50]:
# Leemos todos los archivos en la carpeta
files = os.listdir(dir_insumos)
# Recorremos la carpeta para sacar solo los que tengan extension csv
files_csv = [f for f in files if f[-3:] == 'csv']
# Inicializamos una variable tipo dataframe
df_libor = pd.DataFrame()

# Recorrermos la lista de archivos uno a uno
for files in files_csv:
    path = dir_insumos + files
    # Se lee cada csv con la funcion read y se concatena para dejarlo en un solo df
    data = pd.read_csv(path, delimiter = ',')
    df_libor = df_libor.append(data)
    
df_libor = df_libor[['Benchmark Date', 'Tenor', 'USD ICE LIBOR']]

In [47]:
df_libor_pivot = pd.pivot_table(df_libor, values='USD ICE LIBOR', index='Benchmark Date',
                    columns=['Tenor'], aggfunc=np.sum)

In [51]:
df_libor

,Benchmark Date,Tenor,USD ICE LIBOR
0,"January 2, 2019",Overnight,2.38675
1,"January 2, 2019",1 Week,2.41475
2,"January 2, 2019",1 Month,2.50713
3,"January 2, 2019",2 Month,2.61738
4,"January 2, 2019",3 Month,2.79388
...,...,...,...
93,"April 22, 2021",1 Month,0.10613
94,"April 22, 2021",2 Month,0.15013
95,"April 22, 2021",3 Month,0.17575
96,"April 22, 2021",6 Month,0.21063


In [49]:
df_libor_pivot

Tenor,1 Month,1 Week,1 Year,2 Month,3 Month,6 Month,Overnight
Benchmark Date,,,,,,,
"April 1, 2019",2.49338,2.42125,2.73000,2.56188,2.59550,2.66913,2.38813
"April 1, 2020",1.01625,0.42863,1.00238,1.24450,1.43650,1.19525,0.10875
"April 1, 2021",0.11038,0.08488,0.28050,0.13525,0.19975,0.20125,0.07475
"April 10, 2019",2.48200,2.41425,2.74013,2.54850,2.60350,2.62700,2.38850
"April 11, 2019",2.47263,2.40775,2.73413,2.53675,2.59675,2.63125,2.38850
...,...,...,...,...,...,...,...
"September 6, 2019",2.04900,2.13150,1.94900,2.14450,2.13413,2.03413,2.09475
"September 7, 2020",0.15563,0.10675,0.42100,0.18325,0.24175,0.29325,NaN
"September 8, 2020",0.15550,0.10875,0.42700,0.20413,0.24950,0.30100,0.07938
